In [1]:
%pip install numpy
%pip install pandas
%pip install matplotlib

  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl (15.5 MB)
Note: you may need to restart the kernel to use updated packages.
  Using cached pandas-2.2.1-cp312-cp312-win_amd64.whl.metadata (19 kB)
  Using cached pytz-2024.1-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2024.1-py2.py3-none-any.whl.metadata (1.4 kB)
Using cached pandas-2.2.1-cp312-cp312-win_amd64.whl (11.5 MB)
Using cached pytz-2024.1-py2.py3-none-any.whl (505 kB)
Using cached tzdata-2024.1-py2.py3-none-any.whl (345 kB)
Note: you may need to restart the kernel to use updated packages.
  Using cached matplotlib-3.8.3-cp312-cp312-win_amd64.whl.metadata (5.9 kB)
  Using cached contourpy-1.2.0-cp312-cp312-win_amd64.whl.metadata (5.8 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
     ---------------------------------------- 0.0/162.6 kB ? eta -:--:--
     -- ------------------------------------- 10.2/162.6 kB ? eta -:-

In [ ]:
#;;
#;; Air cargo transport problem.
#;; http://www.inf.unibz.it/~tessaris/teaching/AI_06-07/labs/2007-01-10/planning.html
#;;

#(define (domain air-cargo)
#  (:requirements :strips)
#  (:predicates (In ?obj ?place)
#           (At ?obj ?place)
#           (Cargo ?obj)
#           (Plane ?obj)
#           (Airport ?obj))

#  (:action LOAD
#     :parameters (?c ?p ?a)
#     :precondition (and (At ?c ?a) (At ?p ?a)
#             (Cargo ?c) (Plane ?p) (Airport ?a))
#     :effect (and (In ?c ?p) (not (At ?c ?a))))
#  
#  (:action UNLOAD
#     :parameters (?c ?p ?a)
#     :precondition (and (In ?c ?p) (At ?p ?a)
#             (Cargo ?c) (Plane ?p) (Airport ?a))
#     :effect (and (At ?c ?a) (not (In ?c ?p))))
#  
#  (:action FLY
#     :parameters (?p ?from ?to)
#     :precondition (and (At ?p ?from)
#             (Plane ?p) (Airport ?from) (Airport ?to))
#     :effect (and (At ?p ?to) (not (At ?p ?from))))
#  )

In [44]:
# STRIPS domain - Air Cargo Transportation
from stripsProblem import STRIPS_domain, Strips

def In(obj):
    return obj+'_is_in'

def At(obj):
    return obj+"_is_at"

def Cargo(obj):
    return obj+"_is_cargo"

def Plane(obj):
    return obj+"_is_plane"

def Airport(obj):
    return obj+"_is_airport"

def Load(cargo, plane, airport):
    return "load_"+cargo+"_from_"+airport+"_to_"+plane

def Unload(cargo, plane, airport):
    return "unload_"+cargo+"_from_"+plane+"_to_"+airport

def Fly(plane, airport_from, airport_to):
    return "fly_"+plane+"_from_"+airport_from+"_to_"+airport_to


boolean = {True, False}
null = {'null'}
def create_air_cargo_transportation(cargo={'c1','c2'}, planes={'p1','p2'}, airports={'sfo', 'jfk'}):
    #feature_domain_dict = {Cargo(x):True for x in cargo}
    #feature_domain_dict.update({Cargo(x):False for x in cargo_storage})
    #feature_domain_dict.update({Plane(x):True for x in planes})
    #feature_domain_dict.update({Plane(x):False for x in cargo|airports})
    #feature_domain_dict.update({Airport(x):True for x in airports})
    #feature_domain_dict.update({Airport(x):False for x in cargo|planes})
    feature_domain_dict = {In(x):planes|null for x in cargo}
    feature_domain_dict.update({At(x):airports|null for x in planes|cargo})


    stmap = {Strips(Load(c, p, a), {At(c):a, At(p):a}, {In(c):p, At(c):'null'}) 
                for c in cargo
                for p in planes
                for a in airports}
    stmap.update({Strips(Unload(c, p, a), {In(c):p, At(p):a}, {At(c):a, In(c):'null'})
                  for c in cargo
                  for p in planes
                  for a in airports})
    
    stmap.update({Strips(Fly(p, a_from, a_to), {At(p):a_from}, {At(p):a_to})
                  for p in planes
                  for a_from in airports
                  for a_to in airports
                  if a_from != a_to})
    
    return STRIPS_domain(feature_domain_dict, stmap)



    



In [45]:
from stripsProblem import Planning_problem, STRIPS_domain
# Problem 1

air_cargo1dom = create_air_cargo_transportation()

air_setup1 = {
                At('c1'):'sfo',
                At('c2'):'jfk',
                At('p1'):'sfo',
                At('p2'):'jfk',
                In('c1'):'null',
                In('c2'):'null'}


air_cargo1 = Planning_problem(air_cargo1dom,
                              air_setup1, #initial state
                               {At('c1'):'jfk', At('c2'):'sfo', In('c1'):'null', In('c2'):'null'}) #goal


In [46]:
#test
from searchMPP import SearcherMPP
from stripsForwardPlanner import Forward_STRIPS
SearcherMPP(Forward_STRIPS(air_cargo1)).search()

Solution: {'c1_is_at': 'sfo', 'c2_is_at': 'jfk', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'null'}
   --load_c2_from_jfk_to_p2--> {'c1_is_at': 'sfo', 'c2_is_at': 'null', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'p2'}
   --fly_p2_from_jfk_to_sfo--> {'c1_is_at': 'sfo', 'c2_is_at': 'null', 'p1_is_at': 'sfo', 'p2_is_at': 'sfo', 'c1_is_in': 'null', 'c2_is_in': 'p2'}
   --load_c1_from_sfo_to_p2--> {'c1_is_at': 'null', 'c2_is_at': 'null', 'p1_is_at': 'sfo', 'p2_is_at': 'sfo', 'c1_is_in': 'p2', 'c2_is_in': 'p2'}
   --unload_c2_from_p2_to_sfo--> {'c1_is_at': 'null', 'c2_is_at': 'sfo', 'p1_is_at': 'sfo', 'p2_is_at': 'sfo', 'c1_is_in': 'p2', 'c2_is_in': 'null'}
   --fly_p2_from_sfo_to_jfk--> {'c1_is_at': 'null', 'c2_is_at': 'sfo', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'p2', 'c2_is_in': 'null'}
   --unload_c1_from_p2_to_jfk--> {'c1_is_at': 'jfk', 'c2_is_at': 'sfo', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_

{'c1_is_at': 'sfo', 'c2_is_at': 'jfk', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'null'}
   --load_c2_from_jfk_to_p2--> {'c1_is_at': 'sfo', 'c2_is_at': 'null', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'p2'}
   --fly_p2_from_jfk_to_sfo--> {'c1_is_at': 'sfo', 'c2_is_at': 'null', 'p1_is_at': 'sfo', 'p2_is_at': 'sfo', 'c1_is_in': 'null', 'c2_is_in': 'p2'}
   --load_c1_from_sfo_to_p2--> {'c1_is_at': 'null', 'c2_is_at': 'null', 'p1_is_at': 'sfo', 'p2_is_at': 'sfo', 'c1_is_in': 'p2', 'c2_is_in': 'p2'}
   --unload_c2_from_p2_to_sfo--> {'c1_is_at': 'null', 'c2_is_at': 'sfo', 'p1_is_at': 'sfo', 'p2_is_at': 'sfo', 'c1_is_in': 'p2', 'c2_is_in': 'null'}
   --fly_p2_from_sfo_to_jfk--> {'c1_is_at': 'null', 'c2_is_at': 'sfo', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'p2', 'c2_is_in': 'null'}
   --unload_c1_from_p2_to_jfk--> {'c1_is_at': 'jfk', 'c2_is_at': 'sfo', 'p1_is_at': 'sfo', 'p2_is_at': 'jfk', 'c1_is_in': 'null', 'c2_is_in': 'null

In [13]:
# Problem 2
import stripsProblem

SearcherMPP(Forward_STRIPS(stripsProblem.problem1)).search()

Solution: {'RLoc': 'lab', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mc_lab--> {'RLoc': 'mr', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mc_mr--> {'RLoc': 'cs', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --puc--> {'RLoc': 'cs', 'MW': True, 'SWC': True, 'RHC': True, 'RHM': False}
   --mc_cs--> {'RLoc': 'off', 'MW': True, 'SWC': True, 'RHC': True, 'RHM': False}
   --dc--> {'RLoc': 'off', 'MW': True, 'SWC': False, 'RHC': False, 'RHM': False} (cost: 5)
 13 paths have been expanded and 10 paths remain in the frontier


{'RLoc': 'lab', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mc_lab--> {'RLoc': 'mr', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --mc_mr--> {'RLoc': 'cs', 'MW': True, 'SWC': True, 'RHC': False, 'RHM': False}
   --puc--> {'RLoc': 'cs', 'MW': True, 'SWC': True, 'RHC': True, 'RHM': False}
   --mc_cs--> {'RLoc': 'off', 'MW': True, 'SWC': True, 'RHC': True, 'RHM': False}
   --dc--> {'RLoc': 'off', 'MW': True, 'SWC': False, 'RHC': False, 'RHM': False}

In [ ]:
# Problem 3